In [0]:
table_name = []

for i in dbutils.fs.ls('mnt/silver/SalesLT/'): #Buscar o nome de cada uma das pastas dentro de '/silver'
    table_name.append(i.name.split('/')[0])

In [0]:
table_name #mostrar lista com o nome das pastas

['Address',
 'Customer',
 'CustomerAddress',
 'Product',
 'ProductCategory',
 'ProductDescription',
 'ProductModel',
 'ProductModelProductDescription',
 'SalesOrderDetail',
 'SalesOrderHeader']

In [0]:
from pyspark.sql.functions import col
import os

for name in table_name:
    # Corrected path without '.delta' extension
    path = '/mnt/silver/SalesLT/' + name + '/'
    df = spark.read.format('delta').load(path)

    column_names = df.columns
    
    for old_col_name in column_names:
        new_col_name = "".join(["_" + char if char.isupper() and not old_col_name[i - 1].isupper() else char for i, char in enumerate(old_col_name)]).lstrip("_")

        df = df.withColumnRenamed(old_col_name, new_col_name)

    output_path = '/mnt/gold/SalesLT/' + name + '/'
    df.write.format('delta').mode("overwrite").save(output_path)
